### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import subjects
from joblib import Parallel, delayed
from neuropy import plotting
from neuropy.analyses import Decode1d, Pf1D
from neuropy.analyses.decoders import radon_transform
from neuropy.core import Epoch
from neuropy.utils import signal_process
from neuropy.utils.mathutil import min_max_scaler
from neuropy.utils.position_util import run_direction
from scipy import stats
from tqdm.notebook import tqdm

### Backup of replay files

In [ ]:
sessions = subjects.pf_sess()

# for s, sess in enumerate(sessions):
    
#     sess.replay_pbe.save(sess.filePrefix.with_suffix('.bak.06-13-2022.pbe.replay'))

### Visualize decoding in one session

In [ ]:
sessions = subjects.nsd.ratNday2
for sub, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
    pos = sess.maze
    # pos.t_start = pos.t_start - 0.5
    forward = sess.maze_run["up"]

    pf = Pf1D(
        neurons=neurons,
        position=pos,
        speed_thresh=5,
        sigma=4,
        grid_bin=2,
        epochs=forward,
        frate_thresh=0.1,
    )
    pf_neurons = neurons.get_by_id(pf.neuron_ids)
    epochs = sess.pbe.time_slice(post[0], post[0] + 2 * 3600)
    decode = Decode1d(
        neurons=pf_neurons,
        ratemap=pf,
        epochs=epochs,
        bin_size=0.02,
        score_method="wcorr",
        n_jobs=6,
    )
    decode.calculate_shuffle_score(n_iter=200, method="neuron_id")


In [ ]:
decode.plot_summary()

In [ ]:
from hfuncs import plot_in_bokeh
import bokeh.plotting as bplot

measure = decode.sequence_score 
# measure[np.isnan(measure)] = 0
# bplot.output_file(subjects.figpath_sd/'test_wake_decoding.html')
bplot.output_notebook()
p = plot_in_bokeh(
    x=epochs.starts / 3600,
    y=measure,
    img_arr=decode.posterior,
    color_by=measure,
    palette="jet",
    size=10,
)
# p.line(pos.time,pos.x/350 + 0.2,line_width=2,color='black')
bplot.show(p)


### Save radon scores as separate epoch file

In [ ]:
# sessions = subjects.pf_sess()
sessions = subjects.sd.ratVday2

radon_df = []
for sub, sess in enumerate(tqdm(sessions)):

    replay_pbe = sess.replay_pbe.to_dataframe()
    metadata = sess.replay_pbe.metadata
    starts = sess.replay_pbe.starts
    stops = sess.replay_pbe.stops

    posteriors = [metadata["up_posterior"], metadata["down_posterior"]]

    r_score,velocity = [],[]
    for p in posteriors:
        results = Parallel(n_jobs=6)(
            delayed(radon_transform)(epoch, nlines=20000, dt=0.02, dx=2, neighbours=8)
            for epoch in p
        )
        score, v, intercept = np.asarray(results).T
        r_score.append(score)
        velocity.append(v)

    epoch_df = pd.DataFrame(
        dict(
            start=starts,
            stop=stops,
            up_radon_score=r_score[0],
            up_velocity = velocity[0],
            down_radon_score=r_score[1],
            down_velocity = velocity[1],
            label='pbe',
        )
    )
    Epoch(epochs=epoch_df).save(sess.filePrefix.with_suffix('.pbe.radon'))


### Test radon score distribution with varying neighbours

In [ ]:
# sessions = subjects.pf_sess()
sess = subjects.sd.ratUday1[0]
replay_pbe = sess.replay_radon.to_dataframe()
pbe_epochs = sess.replay_radon.flatten()
neurons = sess.neurons_stable.get_neuron_type(['pyr','mua'])
n_spikes = [np.histogram(_,bins=pbe_epochs)[0][::2] for _ in neurons.spiketrains]
fire_bool = (np.array(n_spikes)>0).sum(axis=0)>=5
replay_pbe = replay_pbe[fire_bool]

metadata = sess.replay_radon.metadata
starts = replay_pbe.start.values
posteriors = metadata["up_posterior"]
posteriors = [posteriors[_] for _ in np.where(fire_bool)[0]]

score_df = []
for e in ["pre", "maze"]:
    period = sess.paradigm[e].flatten()

    if e == "post":
        period = [period[0], period[0] + 2 * 3600]

    indx = np.where((starts > period[0]) & (starts < period[1]))[0]
    e_posteriors = [posteriors[_] for _ in indx]

    for i, neigh in enumerate([8, 25]):
        results = Parallel(n_jobs=6)(
            delayed(radon_transform)(
                epoch, nlines=6000, dt=0.02, dx=2, neighbours=neigh
            )
            for epoch in e_posteriors
        )
        score, v, intercept = np.asarray(results).T

        score_df.append(pd.DataFrame(dict(n=neigh, score=score, zt=e)))

score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
_,ax = plt.subplots()

sns.violinplot(data=score_df,x='n',y='score',hue='zt')

### Test radon score with and without normalizing posterior

In [ ]:

def scale_post(arr):
    return (arr - arr.min()) / (arr.max() - arr.min())


# sessions = subjects.pf_sess()
sess = subjects.sd.ratUday1[0]
replay_pbe = sess.replay_radon.to_dataframe()
pbe_epochs = sess.replay_radon.flatten()
neurons = sess.neurons_stable.get_neuron_type(["pyr", "mua"])
n_spikes = [np.histogram(_, bins=pbe_epochs)[0][::2] for _ in neurons.spiketrains]
fire_bool = (np.array(n_spikes) > 0).sum(axis=0) >= 5
replay_pbe = replay_pbe[fire_bool]

metadata = sess.replay_radon.metadata
starts = replay_pbe.start.values
posteriors = metadata["up_posterior"]
posteriors = [posteriors[_] for _ in np.where(fire_bool)[0]]

score_df = []
for e in ["pre", "maze"]:
    period = sess.paradigm[e].flatten()

    if e == "post":
        period = [period[0], period[0] + 2 * 3600]

    indx = np.where((starts > period[0]) & (starts < period[1]))[0]
    e_posteriors = [posteriors[_] for _ in indx]

    for norm in [True, False]:

        if norm:
            p = [scale_post(_) for _ in e_posteriors]
        else:
            p = e_posteriors
        
        results = Parallel(n_jobs=6)(
            delayed(radon_transform)(epoch, nlines=6000, dt=0.02, dx=2, neighbours=8)
            for epoch in p
        )
        score, v, intercept = np.asarray(results).T

        score_df.append(pd.DataFrame(dict(score=score, norm=norm, zt=e)))

score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
_, ax = plt.subplots()

# sns.violinplot(
#     data=score_df[score_df.norm == True],
#     x="zt",
#     y="score",
#     hue="norm",
#     inner="quartile",
# )

ax.pcolormesh(p[100])


### Test radon score with varying place field peak frate thresh

In [ ]:

sess = subjects.sd.ratUday1[0]
period = sess.paradigm["maze"].flatten()
starts = sess.replay_wcorr.starts

neurons = sess.neurons_stable.get_neuron_type(["pyr"])
# neurons = neurons[neurons.firing_rate<=10]

maze_pos = sess.maze
# maze_run= run_direction(maze_pos,min_distance=30,sigma=0.2,speed_thresh=(8,None))

peak_frates = [0.3,0.5,1]
epochs = sess.get_zt_epochs()
# epochs = epochs[:3]

score_df = []
for b in peak_frates:
    pf = Pf1D(
        neurons=neurons,
        position=sess.maze,
        sigma=4,
        grid_bin=2,
        epochs=sess.maze_run["up"],
        frate_thresh=b,
    )
    pf_neurons = neurons.get_by_id(pf.neuron_ids)
    print(pf_neurons.n_neurons)

    for e in epochs.itertuples():
        pbe_bool = (starts > e.start) & (starts < e.stop)
        pbe_epochs = sess.replay_wcorr[pbe_bool]

        decode = Decode1d(
            neurons=pf_neurons,
            ratemap=pf,
            epochs=pbe_epochs,
            bin_size=0.02,
            n_jobs=6,
        )
        radon = decode.get_radon_transform(nlines=5000,margin=16)[0]
        score_df.append(pd.DataFrame(dict(frate_thresh=b, zt=e.label, score=radon)))

score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
_,ax = plt.subplots()

sns.violinplot(data=score_df,x='zt',y='score',hue='frate_thresh')

### Test radon score with varying track binning size

In [ ]:

sess = subjects.sd.ratUday4[0]
period = sess.paradigm["maze"].flatten()
starts = sess.replay_wcorr.starts

neurons = sess.neurons_stable.get_neuron_type(["pyr",'mua'])
neurons = neurons[neurons.firing_rate<=10]

maze_pos = sess.maze
# maze_run= run_direction(maze_pos,min_distance=30,sigma=0.2,speed_thresh=(8,None))

grid_binsz = [2,2.5,4]
epochs = sess.get_zt_epochs()
# epochs = epochs[:3]

score_df = []
for b in grid_binsz:
    pf = Pf1D(
        neurons=neurons,
        position=sess.maze,
        sigma=4.5,
        grid_bin=b,
        epochs=sess.maze_run["up"],
        frate_thresh=1,
    )
    pf_neurons = neurons.get_by_id(pf.neuron_ids)

    for e in epochs.itertuples():
        pbe_bool = (starts > e.start) & (starts < e.stop)
        pbe_epochs = sess.replay_wcorr[pbe_bool]

        decode = Decode1d(
            neurons=pf_neurons,
            ratemap=pf,
            epochs=pbe_epochs,
            bin_size=0.02,
            n_jobs=6,
        )
        radon = decode.get_radon_transform(nlines=5000,margin=16)[0]
        score_df.append(pd.DataFrame(dict(binsz=b, zt=e.label, score=radon)))

score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
_,ax = plt.subplots()

sns.violinplot(data=score_df,x='zt',y='score',hue='binsz')

### Test radon score with varying binning size

In [ ]:
sess = subjects.sd.ratRday2[0]
period = sess.paradigm["maze"].flatten()
starts = sess.replay_wcorr.starts

neurons = sess.neurons_stable.get_neuron_type(['pyr'])
# neurons = neurons[neurons.firing_rate<=10] 

maze_pos = sess.maze
# maze_run= run_direction(maze_pos,min_distance=30,sigma=0.2,speed_thresh=(8,None))
pf = Pf1D(
    neurons=neurons,
    position=sess.maze,
    sigma=4,
    grid_bin=2,
    epochs=sess.maze_run["up"],
    frate_thresh=0.3,
)
pf_neurons = neurons.get_by_id(pf.neuron_ids)


bin_sizes = [0.02]
epochs = sess.get_zt_epochs()
# epochs = epochs[:3]

score_df = []
for e in epochs.itertuples():
    pbe_bool = (starts > e.start) & (starts < e.stop)
    pbe_epochs = sess.replay_wcorr[pbe_bool]
    for b in bin_sizes:
        decode = Decode1d(
            neurons=pf_neurons,
            ratemap=pf,
            epochs=pbe_epochs,
            bin_size=b,
            score_method="radon_transform",
            radon_kw=dict(nlines=5000, decode_margin=16),
            n_jobs=6,
        )
        score_df.append(pd.DataFrame(dict(binsz=b, zt=e.label, score=decode.score)))

score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
_,ax = plt.subplots()

sns.violinplot(data=score_df,x='binsz',y='score',hue='zt')

### Test radon score with varying number of lines

In [ ]:
sess = subjects.sd.ratNday1[0]
period = sess.paradigm["maze"].flatten()
starts = sess.replay_pbe.starts

neurons = sess.neurons
maze_pos = sess.maze
# maze_run= run_direction(maze_pos,min_distance=30,sigma=0.2,speed_thresh=(8,None))
pf = Pf1D(
    neurons=neurons,
    position=sess.maze,
    sigma=4,
    grid_bin=2,
    epochs=sess.maze_run["up"],
    frate_thresh=0.3,
)
pf_neurons = neurons.get_by_id(pf.neuron_ids)


# nlines = np.arange(2000,30000,10000)
nlines = [5000]
epochs = sess.get_zt_epochs()
epochs = epochs[2]

score_df = []
for e in epochs.itertuples():
    pbe_bool = (starts > e.start) & (starts < e.stop)
    pbe_epochs = sess.pbe[pbe_bool]
    for nl in nlines:
        decode = Decode1d(
            neurons=pf_neurons,
            ratemap=pf,
            epochs=pbe_epochs,
            bin_size=0.02,
            score_method="radon_transform",
            radon_kw=dict(nlines=nl, decode_margin=16),
            n_jobs=6,
        )
        decode.calculate_shuffle_score(method="neuron_id", n_iter=10)
        score_df.append(pd.DataFrame(dict(nlines=nl, zt=e.label, score=decode.score,shuffle_score=decode.shuffle_score.mean(axis=0))))


score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
_,ax = plt.subplots()

# sns.violinplot(data=score_df,x='nlines',y='score',inner='quartile')
sns.violinplot(data=score_df,x='nlines',y='score')
sns.violinplot(data=score_df,x='nlines',y='shuffle_score')

### Test radon score with and without normalizing place fields firing rates
- normalized place fields improves the radon score across the board, but the increase for PRE is higher compared to MAZE and POST.

In [ ]:
sess = subjects.nsd.ratNday2[0]
period = sess.paradigm["maze"].flatten()
starts = sess.replay_pbe.starts

neurons = sess.neurons_stable.get_neuron_type(['pyr','mua'])
maze_pos = sess.maze
# maze_run= run_direction(maze_pos,min_distance=30,sigma=0.2,speed_thresh=(8,None))
pf = Pf1D(
    neurons=neurons,
    position=sess.maze,
    sigma=4,
    grid_bin=2,
    epochs=sess.maze_run["up"],
    frate_thresh=0.3,
)
pf_neurons = neurons.get_by_id(pf.neuron_ids)
og_tc = pf.tuning_curves.copy()
norm_tc = min_max_scaler(og_tc) 

epochs = sess.get_zt_epochs()
epochs = epochs[:3]

# score_df = []
# for e in epochs.itertuples():
#     pbe_bool = (starts > e.start) & (starts < e.stop)
#     pbe_epochs = sess.pbe[pbe_bool]
#     for norm in [False,True]:

#         if norm:
#             pf.tuning_curves = norm_tc 
#         else:
#             pf.tuning_curves = og_tc

#         decode = Decode1d(
#             neurons=pf_neurons,
#             ratemap=pf,
#             epochs=pbe_epochs,
#             bin_size=0.02,
#             score_method="radon_transform",
#             radon_kw=dict(nlines=5000, decode_margin=16),
#             n_jobs=6,
#         )
#         score_df.append(pd.DataFrame(dict(norm=norm, zt=e.label, score=decode.score)))

# score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
_,ax = plt.subplots()

sns.violinplot(data=score_df,x='zt',y='score',hue='norm',split=True,inner='quartile')

In [ ]:
_, ax = plt.subplots()

posteriors = sess.replay_radon_mua.metadata["up_posterior"]
stacked_posteriors = np.hstack(posteriors)
stacked_posteriors = np.apply_along_axis(
        np.convolve, axis=0, arr=stacked_posteriors, v=np.ones(2 * 4 + 1), mode="same"
    )

n_bins = np.array([_.shape[1] for _ in posteriors]).cumsum()

ax.imshow(
    stacked_posteriors,
    aspect="auto",
    cmap="binary",
    interpolation="none",
    vmax=0.8,
    origin="lower",
)
ax.vlines(n_bins, 0, 160)


In [ ]:
new_posts = np.hstack([posteriors[_] for _ in range(2862,2864)])
epochs = sess.replay_radon_mua[2862:2864].as_array()
t = np.concatenate([np.arange(e[0],e[1],0.02)[:-1]+0.01 for e in epochs])
y = np.arange(new_posts.shape[0]) 

_,ax = plt.subplots()
ax.pcolormesh(t,y,new_posts,cmap='binary',shading='auto')

### Test radon score including smaller duration PBEs

In [ ]:

sess = subjects.nsd.ratNday2[0]
period = sess.paradigm["maze"].flatten()

neurons = sess.neurons_stable.get_neuron_type(['pyr','mua'])
neurons = neurons[neurons.firing_rate<=10]
maze_pos = sess.maze
# maze_run= run_direction(maze_pos,min_distance=30,sigma=0.2,speed_thresh=(8,None))
pf = Pf1D(
    neurons=neurons,
    position=sess.maze,
    sigma=4,
    grid_bin=2,
    epochs=sess.maze_run["down"],
    frate_thresh=0.3,
)
pf_neurons = neurons.get_by_id(pf.neuron_ids)
og_tc = pf.tuning_curves.copy()
norm_tc = min_max_scaler(og_tc) 

decode_epochs = sess.pbe
epochs = sess.get_zt_epochs()
# epochs = epochs[:3]

score_df = []

for short in [False,True]:
    if short:
        smth_mua = sess.mua.get_smoothed(sigma=0.02)
        pbe_epochs = detect_pbe_epochs(smth_mua,duration=(0.08,0.5))
    else:
        pbe_epochs = decode_epochs

    starts = pbe_epochs.starts
    print(len(starts))

    for e in epochs.itertuples():
        pbe_bool = (starts > e.start) & (starts < e.stop)
        e_epochs = pbe_epochs[pbe_bool]

        decode = Decode1d(
            neurons=pf_neurons,
            ratemap=pf,
            epochs=e_epochs,
            bin_size=0.02,
            # score_method="wcorr",
            score_method="radon_transform",
            radon_kw=dict(nlines=7000, decode_margin=16),
            n_jobs=6,
        )
        score_df.append(pd.DataFrame(dict(short=short, zt=e.label, score=decode.score)))

score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
_,ax = plt.subplots()

sns.violinplot(data=score_df,x='zt',y='score',hue='short',split=True,inner='quartile')

In [ ]:
_,ax = plt.subplots()

sns.violinplot(data=score_df,x='zt',y='score',hue='short',split=True,inner='quartile')

### Test radon score on smoothed spiketrains
- joblib parallel for radon transform uses a lot of memory (>30 GB) when used on smoothed spiketrain with gaussian kernel of sd 30 ms. Works fine for 5 ms and 10 ms gaussian kernels. Most likely it is because of spikecounts are float64 instead of integer.
- It works fine for wcorr. For gaussian kernels 5,10 and 30 ms, changes in scores were observed across all epochs while keeping relative differences same. Compared to no smoothing, wcorr values decreased for 5 ms but increased for 10 and 30. 

In [ ]:
sess = subjects.sd.ratVday2[0]
period = sess.paradigm["maze"].flatten()

neurons = sess.neurons_stable.get_neuron_type(['pyr','mua'])
neurons = neurons[neurons.firing_rate<=10]
maze_pos = sess.maze
# maze_run= run_direction(maze_pos,min_distance=30,sigma=0.2,speed_thresh=(8,None))
pf = Pf1D(
    neurons=neurons,
    position=sess.maze,
    sigma=4,
    grid_bin=2,
    epochs=sess.maze_run["down"],
    frate_thresh=0.3,
)
pf_neurons = neurons.get_by_id(pf.neuron_ids)

pbe_epochs = sess.pbe
starts = pbe_epochs.starts
epochs = sess.get_zt_epochs()
epochs = epochs[:3]

score_df = []

# memory error for 0.03 in randon transform
for sig in [None,0.005,0.01]:
    print(sig)
    for e in epochs.itertuples():
        pbe_bool = (starts > e.start) & (starts < e.stop)
        e_epochs = pbe_epochs[pbe_bool]

        decode = Decode1d(
            neurons=pf_neurons,
            ratemap=pf,
            epochs=e_epochs,
            bin_size=0.02,
            # sigma=sig, # has been removed from Decode1d
            # score_method="wcorr",
            score_method="radon_transform",
            radon_kw=dict(nlines=7000, decode_margin=16),
            n_jobs=6,
        )
        score_df.append(pd.DataFrame(dict(sigma=str(sig), zt=e.label, score=decode.score)))

score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
_,ax = plt.subplots()

sns.violinplot(data=score_df,x='zt',y='score',hue='sigma',inner='quartile')

### Test radon/wcorr scores with and without merging PBEs
- While visualizing the posterior maps, some replay trajectory appeared to continue over multiple events. Some of these 'continuing' trajectory events were temporally close to each other. So here I tested if I merge close PBE events such ones which are 30 or 50 ms apart. It had marginal effect on total disribution, as number of such merges were very few.
- Even though the increase was very small, it was the most for MAZE epoch.   

In [ ]:
sess = subjects.sd.ratRday2[0]
period = sess.paradigm["maze"].flatten()

neurons = sess.neurons_stable.get_neuron_type(['pyr','mua'])
maze_pos = sess.maze
# maze_run= run_direction(maze_pos,min_distance=30,sigma=0.2,speed_thresh=(8,None))
pf = Pf1D(
    neurons=neurons,
    position=sess.maze,
    sigma=4,
    grid_bin=2,
    epochs=sess.maze_run["up"],
    frate_thresh=0.3,
)
pf_neurons = neurons.get_by_id(pf.neuron_ids)
og_tc = pf.tuning_curves.copy()
norm_tc = min_max_scaler(og_tc) 

decode_epochs = sess.pbe
epochs = sess.get_zt_epochs()
# epochs = epochs[:3]

score_df = []

for merge in [False,True]:
    if merge:
        pbe_epochs = decode_epochs.merge(dt=0.05)
    else:
        pbe_epochs = decode_epochs

    starts = pbe_epochs.starts
    print(len(starts))

    for e in epochs.itertuples():
        pbe_bool = (starts > e.start) & (starts < e.stop)
        e_epochs = pbe_epochs[pbe_bool]

        decode = Decode1d(
            neurons=pf_neurons,
            ratemap=pf,
            epochs=e_epochs,
            bin_size=0.02,
            score_method="wcorr",
            # score_method="radon_transform",
            # radon_kw=dict(nlines=7000, decode_margin=16),
            n_jobs=6,
        )
        score_df.append(pd.DataFrame(dict(merge=merge, zt=e.label, score=decode.score)))

score_df = pd.concat(score_df, ignore_index=True)


In [ ]:
_,ax = plt.subplots()

sns.violinplot(data=score_df,x='zt',y='score',hue='merge',split=True,inner='quartile')

### Test if various measures for jump distance are calculated correctly

In [ ]:
sess = subjects.sd.ratNday1[0]

neurons = sess.neurons_stable.get_neuron_type(["pyr", "mua"])
neurons = neurons[neurons.firing_rate <= 10]
maze_pos = sess.maze

pf = Pf1D(
    neurons=neurons,
    position=sess.maze,
    sigma=4,
    grid_bin=2,
    epochs=sess.maze_run["down"],
    frate_thresh=0.3,
)
pf_neurons = neurons.get_by_id(pf.neuron_ids)

pbe_epochs = sess.pbe
starts = pbe_epochs.starts

period = sess.paradigm["maze"].flatten()
pbe_bool = (starts > period[0]) & (starts < period[1])
e_epochs = pbe_epochs[pbe_bool]

decode = Decode1d(
    neurons=pf_neurons, ratemap=pf, epochs=e_epochs, bin_size=0.02, n_jobs=6
)
wcorr1,mean_jd = decode.get_wcorr(jump_stat='mean')
wcorr2,max_jd = decode.get_wcorr(jump_stat='max')
wcorr3,median_jd = decode.get_wcorr(jump_stat='median')

### Test jump distance calculation for shuffles

In [ ]:
sess = subjects.sd.ratRday2[0]
period = sess.paradigm["maze"].flatten()

neurons = sess.neurons_stable.get_neuron_type(["pyr", "mua"])
neurons = neurons[neurons.firing_rate <= 10]
maze_pos = sess.maze

pf = Pf1D(
    neurons=neurons,
    position=sess.maze,
    sigma=4,
    grid_bin=2,
    epochs=sess.maze_run["down"],
    frate_thresh=0.3,
)
pf_neurons = neurons.get_by_id(pf.neuron_ids)

pbe_epochs = sess.pbe
starts = pbe_epochs.starts
epochs = sess.get_zt_epochs()
epochs = epochs[:3]

score_df = []

pbe_bool = (starts > period[0]) & (starts < period[1])
e_epochs = pbe_epochs[pbe_bool]

decode = Decode1d(
    neurons=pf_neurons, ratemap=pf, epochs=e_epochs, bin_size=0.02, n_jobs=6
)
wcorr,jd = decode.get_wcorr(jump_distance=True)
# radon_score,vel,intercept = decode.get_radon_transform(nlines=5000,margin=16)
# shuffled_measures = decode.get_shuffled_wcorr(n_iter=100,jump_distance=True)


In [ ]:
_,ax = plt.subplots()

jd_bins = np.linspace(0,1,100)
shuffle_jd = shuffled_measures[:,0,:].flatten()

hist_shuffle = np.histogram(np.abs(shuffle_jd),jd_bins)[0]
hist_events = np.histogram(np.abs(wcorr),jd_bins)[0]
ax.plot(hist_shuffle/hist_shuffle.sum())
ax.plot(hist_events/hist_events.sum())

### Test continuous trajectory length with respect to shuffles


In [ ]:
from replay_funcs import get_max_length
from neuropy.analyses.decoders import column_shift

sess = subjects.nsd.ratNday2[0]
replay_df = sess.replay_filtered.to_dataframe()
posteriors = replay_df['posterior'].to_list()

arr = posteriors[2557]
length = get_max_length(arr)

sh_lengths = np.zeros(1000) 
for i in range(1000):
    sh_lengths[i] = get_max_length(column_shift(arr))

bins = np.arange(11)
hist_sh = np.histogram(sh_lengths,bins)[0]

_,axs = plt.subplots(1,2)
axs[0].pcolormesh(arr)
axs[1].plot(bins[:-1],hist_sh)
axs[1].axvline(length)


### scratch code for pooled randon transform

In [ ]:
nbins = [_.shape[1] for _ in decode.posterior[:60]]
post = np.hstack(decode.posterior[:60])
arr = post
nlines = 10000
dt = 1
dx = 1
neighbours = 1
t = np.concatenate([np.arange(0, _) for _ in nbins])
max_t = max(nbins)
nt = len(t)
tmid = (nt + 1) / 2 - 1

pos = np.arange(arr.shape[0])
npos = len(pos)
pmid = (npos + 1) / 2 - 1

# using convolution to sum neighbours
arr = np.apply_along_axis(
    np.convolve, axis=0, arr=arr, v=np.ones(2 * neighbours + 1), mode="same"
)

# exclude stationary events by choosing phi little below 90 degree
# NOTE: angle of line is given by (90-phi), refer Kloosterman 2012
phi = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=nlines)
diag_len = np.sqrt((max_t - 1) ** 2 + (npos - 1) ** 2)
rho = np.random.uniform(low=-diag_len / 2, high=diag_len / 2, size=nlines)

rho_mat = np.tile(rho, (nt, 1)).T
phi_mat = np.tile(phi, (nt, 1)).T
t_mat = np.tile(t, (nlines, 1))
posterior = np.zeros((nlines, nt))

y_line = ((rho_mat - (t_mat - tmid) * np.cos(phi_mat)) / np.sin(phi_mat)) + pmid
y_line = np.rint(y_line).astype("int")

# if line falls outside of array in a given bin, replace that with median posterior value of that bin across all positions
t_out = np.where((y_line < 0) | (y_line > npos - 1))
t_in = np.where((y_line >= 0) & (y_line <= npos - 1))
posterior[t_out] = np.median(arr[:, t_out[1]], axis=0)
posterior[t_in] = arr[y_line[t_in], t_in[1]]

# old_settings = np.seterr(all="ignore")
posterior_mean = np.nanmean(posterior, axis=1)

best_line = np.argmax(posterior_mean)
score = posterior_mean[best_line]
best_phi, best_rho = phi[best_line], rho[best_line]
time_mid, pos_mid = nt * dt / 2, npos * dx / 2

velocity = dx / (dt * np.tan(best_phi))
intercept = (
    (dx * time_mid) / (dt * np.tan(best_phi))
    + (best_rho / np.sin(best_phi)) * dx
    + pos_mid
)
# np.seterr(**old_settings)

# return score, -velocity, intercept


### Temporal order of spiking activity compared between every pair of PBE (NSD vs SD)

In [ ]:
sessions = subjects.pf_sess()

pair_corr = []
for s, sess in enumerate(sessions):

    neurons = sess.neurons_stable.get_neuron_type("pyr")
    pbe_all = sess.pbe.flatten()
    n_spikes = [np.histogram(_,bins=pbe_all)[0][::2] for _ in neurons.spiketrains]
    spk_bool = (np.array(n_spikes)>0).sum(axis=0)
    good_pbes = spk_bool>5
    pbe_all = sess.pbe[good_pbes]

    epochs = sess.get_zt_epochs()
    epochs = epochs['2.5-5']
    for e in epochs.itertuples():

        pbe = pbe_all.time_slice(e.start, e.stop)
        pbe_bins = pbe.flatten()
        pbe_durs = pbe.durations
        pbe_starts = pbe.starts

        mean_spike = np.array(
            [
                stats.binned_statistic(_, _, bins=pbe_bins,statistic='median')[0][::2]
                for _ in neurons.spiketrains
            ]
        )
        norm_spike = (mean_spike - pbe_starts[np.newaxis, :]) / pbe_durs
        df = pd.DataFrame(data=norm_spike)
        pbe_corr = np.asarray(df.corr(method='spearman'))
        np.fill_diagonal(pbe_corr,0)
        pbe_corr_flat = pbe_corr[np.tril_indices_from(pbe_corr,k=-1)]
        pbe_corr_flat = pbe_corr_flat[~np.isnan(pbe_corr_flat)]
    
        pair_corr.append(pd.DataFrame(dict(corr=pbe_corr_flat,zt=e.label,grp=sess.tag)))

pair_corr = pd.concat(pair_corr,ignore_index=True)

In [ ]:
from neuropy import plotting
_,axs = plt.subplots(1,4)

for i,p in enumerate([0,116,1331,1320]):
    e = pbe[p].flatten()
    plotting.plot_raster(neurons.time_slice(*e),ax=axs[i])

In [ ]:
from scipy.ndimage import gaussian_filter
_,ax = plt.subplots()

# mat = pbe_corr.copy()
# mat[np.isnan(mat)] = 0
# mat = gaussian_filter(mat,sigma=2)
sns.violinplot(data=pair_corr,x='zt',y='corr',hue='grp',split=True)
# ax.imshow(mat)

### Test different shuffling procedures

In [ ]:
from neuropy.analyses.decoders import wcorr
import pingouin as pg

sessions = subjects.nsd.ratVday1

rng = np.random.default_rng()

def rand_circ_shift(arr):
    ny,nx = arr.shape[0],arr.shape[1]
    shifts = rng.integers(-ny,ny,nx,endpoint=True)
    rand_arr = [np.roll(arr[:,i],shifts[i]) for i in range(nx)]
    return np.array(rand_arr).T

dcorr_df = []
for s, sess in enumerate(sessions):

    neurons = sess.neurons_stable.get_neuron_type('pyr') 
    # spikes = np.concatenate(neurons.spiketrains)
    pbe_epochs = sess.pbe.flatten()

    # fire_bool = [np.all(np.histogram(spikes,bins=np.arange(_[0],_[1]))[0]>0)]
    n_spikes = [np.histogram(_,bins=pbe_epochs)[0][::2] for _ in neurons.spiketrains]
    fire_bool = (np.array(n_spikes)>0).sum(axis=0)>5

    starts = sess.replay_pbe.starts[fire_bool]
    up_posterior = sess.replay_pbe.metadata["up_posterior"]
    up_posterior = [up_posterior[i] for i in range(len(fire_bool)) if fire_bool[i]]
    shuffles = sess.replay_pbe.metadata['up_shuffle_score'][:,fire_bool]


    # evt_indx = [3298,3800]
    evt_indx = rng.choice(len(starts),5)

    _,axs = plt.subplots(len(evt_indx),2,**dict(constrained_layout=True))

    bins = np.arange(-1,1,0.01)
    for i,indx in enumerate(evt_indx):  
        arr=up_posterior[indx]
        corr = wcorr(arr)
        rand_corr = [wcorr(rand_circ_shift(arr)) for _ in range(1000)]

        pos_dist = np.histogram(rand_corr,bins)[0]
        id_dist = np.histogram(shuffles[:,indx],bins)[0]

        axs[i,0].pcolormesh(arr,cmap='binary')
        axs[i,0].set_title(indx)
        axs[i,1].plot(bins[:-1],pos_dist,color='r')
        axs[i,1].plot(bins[:-1],id_dist,color='g')
        axs[i,1].axvline(corr,color='k')
        axs[i,1].set_title(corr.round(2))


In [ ]:
a = [1,2,3,4]

a[[True,False,True,False]]

### Test simple linear regression for posteriors

In [ ]:
from neuropy.analyses.decoders import wcorr
import pingouin as pg

sessions = subjects.pf_sess()

def linfit(arr):
    # pos = np.argmax(arr,axis=0)*2
    pos_mat = np.tile(np.arange(arr.shape[0])[:,np.newaxis],(1,arr.shape[1]))
    pos = np.average(pos_mat,axis=0,weights=arr)*2
    t = np.arange(arr.shape[1])*0.02
    fit = stats.linregress(t,pos)
    ss_res = ((pos - (fit.slope*t + fit.intercept))**2).sum()
    ss_tot = ((pos - pos.mean())**2).sum()
    r2 = 1 - (ss_res/ss_tot)
    return fit.rvalue, r2


score_df = []
for s, sess in enumerate(sessions):

    starts = sess.replay_wcorr_mua.starts
    up_posterior = sess.replay_pbe.metadata["up_posterior"]
    down_posterior = sess.replay_pbe.metadata["down_posterior"]
    shuffles = sess.replay_pbe.metadata['up_shuffle_score']

    up_corr,up_r2 = np.array([linfit(_) for _ in up_posterior]).T
    down_corr,down_r2 = np.array([linfit(_) for _ in down_posterior]).T

    best_bool = up_r2 > down_r2
    corr = np.zeros(len(starts))
    corr[best_bool] = up_corr[best_bool]
    corr[~best_bool] = -down_corr[~best_bool]

    r2 = np.zeros(len(starts))
    r2[best_bool] = up_r2[best_bool]
    r2[~best_bool] = down_r2[~best_bool]



    epochs = sess.get_zt_epochs()
    pre = epochs['PRE'].flatten()
    pre_indx = (starts > pre[0]) & (starts < pre[1])
    mean_pre = r2[pre_indx].mean()

    for i, e in enumerate(epochs.itertuples()):
        indx = (starts > e.start) & (starts < e.stop)

        df = pd.DataFrame(
            dict(
                zt=e.label,
                corr = np.abs(corr[indx]),
                r2=r2[indx]/mean_pre,
                session=s,
                grp=sess.tag,
            )
        )
        score_df.append(df)
    
score_df = pd.concat(score_df,ignore_index=True)

In [ ]:
_,ax = plt.subplots()

sns.violinplot(data= score_df,x='zt',y='corr',hue='grp',split=True,inner='quartile',scale_hue=False)